In [2]:
#Extract

from sqlalchemy import false
from pycoingecko import CoinGeckoAPI
import pandas as pd
import datetime as dt 

cg = CoinGeckoAPI()

class Extract():

    @staticmethod
    def coins():
        try:
            df = pd.DataFrame(cg.get_coins_list(), columns=['id','name','symbol'])
            return df
        except:
            return False
    
    @staticmethod
    def market():
        try:
            df = pd.DataFrame(cg.get_coins_markets(vs_currency='usd'), columns=['id','coinid','current_price','market_cap','price_change_percentage_24h','market_cap_change_percentage_24h','last_updated'])
            return df
        except:
            return False

    @staticmethod
    def trending():
        li=[]
        try:
            trend = cg.get_search_trending()
            for item in trend['coins']:
                li.append(item['item'])
            df = pd.DataFrame(li)
            return df
        except:
            return False

    @staticmethod
    def coins_history(number_of_days):
        try:
            list_of_coins= ['bitcoin','litecoin','ethereum']
            list_his=[]
            start_date = pd.to_datetime("today") - pd.Timedelta(number_of_days, unit='D')
            for coin in list_of_coins:
                day = start_date
                while day.date() <= (pd.to_datetime("today").date()):
                    data = cg.get_coin_history_by_id(id=coin,date=day.date().strftime("%d-%m-%Y"), localization='false')
                    data['date']= day.date().strftime("%d-%m-%Y")
                    list_his.append(data)
                    day = day + pd.Timedelta(1, unit='D')
            
            df = pd.DataFrame(list_his)
            df=df[['id','symbol','name','market_data','date']]
            df['current_price'] = pd.json_normalize(df['market_data'])['current_price.usd']
            df['market_cap'] = pd.json_normalize(df['market_data'])['market_cap.usd']
            df.drop(['market_data'], axis = 1)
            df['ingestion_date'] = pd.to_datetime("today").date().strftime("%d-%m-%Y")
            
            return df
        except:
            return False

In [ ]:
#env var

db_user="postgres"
db_password="postgres"
db_server_name="localhost"
db_database_name="crypto"

In [ ]:
#database

from sqlalchemy.engine import URL
from sqlalchemy import create_engine


connection_url = URL.create(
    drivername = "postgresql+pg8000", 
    username = db_user,
    password = db_password,
    host = db_server_name, 
    port = 5433, 
    database = db_database_name, 
)

engine = create_engine(connection_url)

In [ ]:
#Load

from sqlalchemy import Integer, String, Float, JSON , DateTime, Boolean, BigInteger, Numeric
from sqlalchemy import Table, Column, Integer, String, MetaData, Float, JSON 
from sqlalchemy.dialects import postgresql
import logging


def overwrite_to_database(df: pd.DataFrame,table_name:str,engine)->bool:
    logging.info(f"Writing to table: {table_name}")
    df.to_sql(name=table_name, con=engine, if_exists="replace", index=False)
    logging.info(f"Successful write to table: {table_name}, rows inserted/updated: {len(df)}")
    
# def upsert((df: pd.DataFrame,table_name:str,key_columns: str,,ngine)->bool:
    

In [ ]:
#Extrac_Load_pipeline

def run(tablename):
    
    if table_name == 'coins':
#         df = Extract.coins()
#     elif table_name == 'market':
#         df = Extract.market()
#     elif table_name == 'trending':
#         df = Extract.trending()
    elif table_name == 'coins_history':
        df = Extract.coins_history(1)
    
    overwrite_to_database(df,tablename,engine)

In [ ]:
#main pipeline

tables = ['bitcoin','litecoin','ethereum','solana' ,'umee','terra-luna','evmos','dejitaru-tsuka','reserve-rights-token','insights-network']
for table_name in tables:
    run(table_name)

In [ ]:
from sqlalchemy import Integer, String, Float, JSON , DateTime, Boolean, BigInteger, Numeric


def get_sqlalchemy_column(column_name:str , source_datatype:str, primary_key:bool=False)->Column:
    """
    A helper function that returns a SQLAlchemy column by mapping a pandas dataframe datatypes to sqlalchemy datatypes 
    """
    dtype_map = {
        "int64": BigInteger, 
        "object": String, 
        "datetime64[ns]": DateTime, 
        "float64": Numeric,
        "bool": Boolean
    }
    column = Column(column_name, dtype_map[source_datatype], primary_key=primary_key) 
    return column

def generate_sqlalchemy_schema(df: pd.DataFrame, key_columns:list, table_name, meta): 
    """
    Generates a sqlalchemy table schema that shall be used to create the target table and perform insert/upserts. 
    """
    schema = []
    for column in [{"column_name": col[0], "source_datatype": col[1]} for col in zip(df.columns, [dtype.name for dtype in df.dtypes])]:
        schema.append(get_sqlalchemy_column(**column, primary_key=column["column_name"] in key_columns))
    return Table(table_name, meta, *schema)

In [ ]:
list_of_coins= ['bitcoin','litecoin','ethereum']
list_his=[]
start_date = pd.to_datetime("today") - pd.Timedelta(1, unit='D')
for coin in list_of_coins:
    day = start_date
    while day.date() <= (pd.to_datetime("today").date()):
        data = cg.get_coin_history_by_id(id=coin,date=day.date().strftime("%d-%m-%Y"), localization='false')
        data['price_date']= day.date().strftime("%d-%m-%Y")
        list_his.append(data)
        day = day + pd.Timedelta(1, unit='D')
list_his

In [5]:
import pandas as pd
import datetime

df = pd.DataFrame(list_his)

df['id_date']= df['id'].astype(str)+'_'+df['price_date'].astype(str)

df=df[['id_date','id','symbol','name','market_data','price_date']]

df['current_price'] = pd.json_normalize(df['market_data'])['current_price.usd']

df['market_cap'] = pd.json_normalize(df['market_data'])['market_cap.usd']

df = df.drop(['market_data','symbol','name'], axis = 1)

df['ingestion_date'] = pd.to_datetime("today").date().strftime("%d-%m-%Y")

display(df)

,id_date,id,price_date,current_price,market_cap,ingestion_date
0,bitcoin_03-10-2022,bitcoin,03-10-2022,19065.153837,3.661959e+11,04-10-2022
1,bitcoin_04-10-2022,bitcoin,04-10-2022,19620.567544,3.757679e+11,04-10-2022
2,litecoin_03-10-2022,litecoin,03-10-2022,51.897575,3.692364e+09,04-10-2022
3,litecoin_04-10-2022,litecoin,04-10-2022,54.214857,3.868706e+09,04-10-2022
4,ethereum_03-10-2022,ethereum,03-10-2022,1277.901345,1.549348e+11,04-10-2022
5,ethereum_04-10-2022,ethereum,04-10-2022,1323.504018,1.599511e+11,04-10-2022


In [3]:
list_of_coins= ['bitcoin','litecoin','ethereum']
list_his=[]
start_date = pd.to_datetime("today") - pd.Timedelta(1, unit='D')

In [4]:
for coin in list_of_coins:
    day = start_date
    while day.date() <= (pd.to_datetime("today").date()):
        data = cg.get_coin_history_by_id(id=coin,date=day.date().strftime("%d-%m-%Y"), localization='false')
        data['price_date']= day.date().strftime("%d-%m-%Y")
        list_his.append(data)
        day = day + pd.Timedelta(1, unit='D')
list_his

[{'id': 'bitcoin',
  'symbol': 'btc',
  'name': 'Bitcoin',
  'image': {'thumb': 'https://assets.coingecko.com/coins/images/1/thumb/bitcoin.png?1547033579',
   'small': 'https://assets.coingecko.com/coins/images/1/small/bitcoin.png?1547033579'},
  'market_data': {'current_price': {'aed': 70026.31004384802,
    'ars': 2809486.8258114746,
    'aud': 29696.512766819815,
    'bch': 166.43926973721005,
    'bdt': 1932249.31864759,
    'bhd': 7193.07282606409,
    'bmd': 19065.153837148988,
    'bnb': 66.92648044451008,
    'brl': 103240.0005208526,
    'btc': 1.0,
    'cad': 26284.70816189279,
    'chf': 18797.23123027544,
    'clp': 18439558.458455954,
    'cny': 135667.63470515187,
    'czk': 478441.942058636,
    'dkk': 144802.87475260638,
    'dot': 3086.883535223113,
    'eos': 16426.2940244887,
    'eth': 14.90717007864221,
    'eur': 19470.860310803437,
    'gbp': 17125.751063064978,
    'hkd': 149657.26328777475,
    'huf': 8216123.775524871,
    'idr': 290319797.9320449,
    'ils': 

In [6]:
def coins_history(number_of_days):
        try:
            list_of_coins= ['bitcoin','litecoin','ethereum','solana' ,'umee','terra-luna','evmos','dejitaru-tsuka','reserve-rights-token','insights-network']
            list_his=[]
            start_date = pd.to_datetime("today") - pd.Timedelta(number_of_days, unit='D')
            for coin in list_of_coins:
                day = start_date
                while day.date() <= (pd.to_datetime("today").date()):
                    data = cg.get_coin_history_by_id(id=coin,date=day.date().strftime("%d-%m-%Y"), localization='false')
                    data['price_date']= day.date().strftime("%d-%m-%Y")
                    list_his.append(data)
                    day = day + pd.Timedelta(1, unit='D')
            
            df = pd.DataFrame(list_his)
            df['id_date']= df['id'].astype(str)+'_'+df['price_date'].astype(str)
            df=df[['id_date','id','symbol','name','market_data','price_date']]
            df['current_price'] = pd.json_normalize(df['market_data'])['current_price.usd']
            df['market_cap'] = pd.json_normalize(df['market_data'])['market_cap.usd']
            df = df.drop(['market_data','symbol','name'], axis = 1)
            df['ingestion_date'] = pd.to_datetime("today").date().strftime("%d-%m-%Y")
            
            return df
        except:
            return False
        
coins_history(1)

,id_date,id,price_date,current_price,market_cap,ingestion_date
0,bitcoin_03-10-2022,bitcoin,03-10-2022,19065.153837,3.661959e+11,04-10-2022
1,bitcoin_04-10-2022,bitcoin,04-10-2022,19620.567544,3.757679e+11,04-10-2022
2,litecoin_03-10-2022,litecoin,03-10-2022,51.897575,3.692364e+09,04-10-2022
3,litecoin_04-10-2022,litecoin,04-10-2022,54.214857,3.868706e+09,04-10-2022
4,ethereum_03-10-2022,ethereum,03-10-2022,1277.901345,1.549348e+11,04-10-2022
5,ethereum_04-10-2022,ethereum,04-10-2022,1323.504018,1.599511e+11,04-10-2022
6,solana_03-10-2022,solana,03-10-2022,32.097025,1.139515e+10,04-10-2022
7,solana_04-10-2022,solana,04-10-2022,32.983933,1.173534e+10,04-10-2022
8,umee_03-10-2022,umee,03-10-2022,0.012496,2.889751e+07,04-10-2022
9,umee_04-10-2022,umee,04-10-2022,0.012706,2.941012e+07,04-10-2022


In [ ]:
# overwrite_to_database(df,'coins_history',engine)

In [ ]:
if len(df) > 0 :
    key_columns = Load.get_key_columns(table='coins_history', path='D:\DataEngineering Bootcamp\Project1\project-one\src\crypto\models\extract')
    print(key_columns)
    meta = MetaData()
    table_schema = Load.generate_sqlalchemy_schema(df=df, key_columns=key_columns,table_name='coins_history', meta=meta)
    print(table_schema)
    meta.create_all(engine)
    insert_statement = postgresql.insert(table_schema).values(df.to_dict(orient='records'))
#     print(insert_statement)
    upsert_statement = insert_statement.on_conflict_do_update(
                index_elements=key_columns,
                set_={c.key: c for c in insert_statement.excluded if c.key not in key_columns})
    print(upsert_statement)
    result = engine.execute(upsert_statement)
    print(f"Insert/updated rows: {result.rowcount}")

In [ ]:
from sqlalchemy import Integer, String, Float, JSON , DateTime, Boolean, BigInteger, Numeric
from sqlalchemy import Table, Column, Integer, String, MetaData, Float, JSON 
import jinja2 as j2 
import pandas as pd
import numpy as np
import logging 
from sqlalchemy.dialects import postgresql

class Load():

    @staticmethod
    def get_key_columns(table:str, path:str="extract_queries")->list: 
        """
        get a list of key columns from the .sql file. 
        - `table`: name of the sql file without .sql 
        - `path`: path to the sql file 
        """
        # read sql contents into a variable 
        with open(f"{path}/{table}.sql") as f: 
            raw_sql = f.read()
        try: 
            key_columns = j2.Template(raw_sql).make_module().config["key_columns"] # get key columns 
            return key_columns
        except:
            return []
    
    @staticmethod
    def get_sqlalchemy_column(column_name:str , source_datatype:str, primary_key:bool=False)->Column:
        """
        A helper function that returns a SQLAlchemy column by mapping a pandas dataframe datatypes to sqlalchemy datatypes 
        """
        dtype_map = {
            "int64": BigInteger, 
            "object": String, 
            "datetime64[ns]": DateTime, 
            "float64": Numeric,
            "bool": Boolean
        }
        column = Column(column_name, dtype_map[source_datatype], primary_key=primary_key) 
        return column

    @staticmethod
    def generate_sqlalchemy_schema(df: pd.DataFrame, key_columns:list, table_name, meta): 
        """
        Generates a sqlalchemy table schema that shall be used to create the target table and perform insert/upserts. 
        """
        schema = []
        for column in [{"column_name": col[0], "source_datatype": col[1]} for col in zip(df.columns, [dtype.name for dtype in df.dtypes])]:
            schema.append(Load.get_sqlalchemy_column(**column, primary_key=column["column_name"] in key_columns))
        return Table(table_name, meta, *schema)

    @staticmethod
    def upsert_in_chunks(df:pd.DataFrame, engine, table_schema:Table, key_columns:list, chunksize:int=1000)->bool:
        """
        performs the upsert with several rows at a time (i.e. a chunk of rows). this is better suited for very large sql statements that need to be broken into several steps. 
        """
        max_length = len(df)
        df = df.replace({np.nan: None})
        for i in range(0, max_length, chunksize):
            if i + chunksize >= max_length: 
                lower_bound = i
                upper_bound = max_length 
            else: 
                lower_bound = i 
                upper_bound = i + chunksize
            insert_statement = postgresql.insert(table_schema).values(df.iloc[lower_bound:upper_bound].to_dict(orient='records'))
            upsert_statement = insert_statement.on_conflict_do_update(
                index_elements=key_columns,
                set_={c.key: c for c in insert_statement.excluded if c.key not in key_columns})
            logging.info(f"Inserting chunk: [{lower_bound}:{upper_bound}] out of index {max_length}")
            result = engine.execute(upsert_statement)
        return True 

    @staticmethod
    def upsert_all(df:pd.DataFrame, engine, table_schema:Table, key_columns:list)->bool:
        """
        performs the upsert with all rows at once. this may cause timeout issues if the sql statement is very large. 
        """
        insert_statement = postgresql.insert(table_schema).values(df.to_dict(orient='records'))
        upsert_statement = insert_statement.on_conflict_do_update(
            index_elements=key_columns,
            set_={c.key: c for c in insert_statement.excluded if c.key not in key_columns})
        result = engine.execute(upsert_statement)
        logging.info(f"Insert/updated rows: {result.rowcount}")
        return True 

    @staticmethod
    def upsert_to_database(df: pd.DataFrame, table_name: str, key_columns: str, engine, chunksize:int=1000)->bool: 
        """
        Upsert dataframe to a database table 
        - `df`: pandas dataframe 
        - `table`: name of the target table 
        - `key_columns`: name of key columns to be used for upserting 
        - `engine`: connection engine to database 
        - `chunksize`: if chunksize greater than 0 is specified, then the rows will be inserted in the specified chunksize. e.g. 1000 rows at a time. 
        """
        meta = MetaData()
        logging.info(f"Generating table schema: {table_name}")
        table_schema = Load.generate_sqlalchemy_schema(df=df, key_columns=key_columns,table_name=table_name, meta=meta)
        meta.create_all(engine)
        logging.info(f"Table schema generated: {table_name}")
        logging.info(f"Writing to table: {table_name}")
        if chunksize > 0:
            Load.upsert_in_chunks(df=df, engine=engine, table_schema=table_schema, key_columns=key_columns, chunksize=chunksize)
        else: 
            Load.upsert_all(df=df, engine=engine, table_schema=table_schema, key_columns=key_columns)
        logging.info(f"Successful write to table: {table_name}")
        return True 

    @staticmethod
    def overwrite_to_database(df: pd.DataFrame, table_name: str, engine)->bool: 
        """
        Upsert dataframe to a database table 
        - `df`: pandas dataframe 
        - `table`: name of the target table 
        - `engine`: connection engine to database 
        """
        logging.info(f"Writing to table: {table_name}")
        df.to_sql(name=table_name, con=engine, if_exists="replace", index=False)
        logging.info(f"Successful write to table: {table_name}, rows inserted/updated: {len(df)}")
        return True 

In [ ]:
def get_sqlalchemy_column(column_name:str , source_datatype:str, primary_key:bool=False)->Column:
        """
        A helper function that returns a SQLAlchemy column by mapping a pandas dataframe datatypes to sqlalchemy datatypes 
        """
        dtype_map = {
            "int64": BigInteger, 
            "object": String, 
            "datetime64[ns]": DateTime, 
            "float64": Numeric,
            "bool": Boolean
        }
        column = Column(column_name, dtype_map[source_datatype], primary_key=primary_key) 
        return column

In [ ]:
def generate_sqlalchemy_schema(df: pd.DataFrame, key_columns:list, table_name, meta): 
        """
        Generates a sqlalchemy table schema that shall be used to create the target table and perform insert/upserts. 
        """
        schema = []
        for column in [{"column_name": col[0], "source_datatype": col[1]} for col in zip(df.columns, [dtype.name for dtype in df.dtypes])]:
            schema.append(Load.get_sqlalchemy_column(**column, primary_key=column["column_name"] in key_columns))
        return Table(table_name, meta, *schema)

In [ ]:
def upsert_in_chunks(df:pd.DataFrame, engine, table_schema:Table, key_columns:list, chunksize:int=1000)->bool:
        """
        performs the upsert with several rows at a time (i.e. a chunk of rows). this is better suited for very large sql statements that need to be broken into several steps. 
        """
        max_length = len(df)
        df = df.replace({np.nan: None})
        for i in range(0, max_length, chunksize):
            if i + chunksize >= max_length: 
                lower_bound = i
                upper_bound = max_length 
            else: 
                lower_bound = i 
                upper_bound = i + chunksize
            insert_statement = postgresql.insert(table_schema).values(df.iloc[lower_bound:upper_bound].to_dict(orient='records'))
            upsert_statement = insert_statement.on_conflict_do_update(
                index_elements=key_columns,
                set_={c.key: c for c in insert_statement.excluded if c.key not in key_columns})
            logging.info(f"Inserting chunk: [{lower_bound}:{upper_bound}] out of index {max_length}")
            result = engine.execute(upsert_statement)
        return True 

In [ ]:
def upsert_all(df:pd.DataFrame, engine, table_schema:Table, key_columns:list)->bool:
        """
        performs the upsert with all rows at once. this may cause timeout issues if the sql statement is very large. 
        """
        insert_statement = postgresql.insert(table_schema).values(df.to_dict(orient='records'))
        upsert_statement = insert_statement.on_conflict_do_update(
            index_elements=key_columns,
            set_={c.key: c for c in insert_statement.excluded if c.key not in key_columns})
        result = engine.execute(upsert_statement)
        logging.info(f"Insert/updated rows: {result.rowcount}")
        return True 

In [ ]:
def upsert_to_database(df: pd.DataFrame, table_name: str, key_columns: str, engine, chunksize:int=1000)->bool: 
        """
        Upsert dataframe to a database table 
        - `df`: pandas dataframe 
        - `table`: name of the target table 
        - `key_columns`: name of key columns to be used for upserting 
        - `engine`: connection engine to database 
        - `chunksize`: if chunksize greater than 0 is specified, then the rows will be inserted in the specified chunksize. e.g. 1000 rows at a time. 
        """
        meta = MetaData()
        logging.info(f"Generating table schema: {table_name}")
        table_schema = Load.generate_sqlalchemy_schema(df=df, key_columns=key_columns,table_name=table_name, meta=meta)
        meta.create_all(engine)
        logging.info(f"Table schema generated: {table_name}")
        logging.info(f"Writing to table: {table_name}")
        if chunksize > 0:
            Load.upsert_in_chunks(df=df, engine=engine, table_schema=table_schema, key_columns=key_columns, chunksize=chunksize)
        else: 
            Load.upsert_all(df=df, engine=engine, table_schema=table_schema, key_columns=key_columns)
        logging.info(f"Successful write to table: {table_name}")